# YOLOV3 training example


In [1]:
from yolo import YOLO, detect_video
from PIL import Image
import matplotlib.pyplot as plt
from yolo3.model import yolo_eval, yolo_body, tiny_yolo_body
from keras.layers import Input
import numpy as np
import keras.backend as K
from keras.layers import Input, Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

from yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss
from yolo3.utils import get_random_data
import tensorflow as tf
from keras.utils import multi_gpu_model
from keras.models import load_model

Using TensorFlow backend.
/home/myx/anaconda3/envs/cs231n/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def get_classes(classes_path):
    '''loads the classes'''
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)

def create_model(input_shape, anchors, num_classes, load_pretrained=True, freeze_body=2,
            weights_path='model_data/yolo.h5'):
    '''create the training model'''
   
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)

    y_true = [Input(shape=(h//{0:32, 1:16, 2:8}[l], w//{0:32, 1:16, 2:8}[l], \
        num_anchors//3, num_classes+5)) for l in range(3)]

    model_body = yolo_body(image_input, num_anchors//3, num_classes)
    print('Create YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))

    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
        print('Load weights {}.'.format(weights_path))
        if freeze_body in [1, 2]:
            # Freeze darknet53 body or freeze all but 3 output layers.
            num = (185, -3)[freeze_body-1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))

    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.5})(
        [*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss)

    return model

def data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes):
    '''data generator for fit_generator'''
    n = len(annotation_lines)
    #print("annotation_lines: ", n)
    i = 0
    while True:
        image_data = []
        box_data = []
        for b in range(batch_size):
           # if i==0:
               # np.random.shuffle(annotation_lines)
            #随机裁剪、旋转、变换颜色（hue）、变换饱和度（saturation）， 变换曝光度（exposure shifts）
            # random 为 False的时候会出错  导致loss 为 non
            image, box = get_random_data(annotation_lines[i], input_shape, random=True)
            image_data.append(image)
            box_data.append(box)
            i = (i+1) % n
        image_data = np.array(image_data)
        box_data = np.array(box_data)
        # 每一个grid cell内的box是否含有object 以及如果含有的情况下的具体box信息
        y_true = preprocess_true_boxes(box_data, input_shape, anchors, num_classes)
        yield [image_data, *y_true], np.zeros(batch_size)

def data_generator_wrapper(annotation_lines, batch_size, input_shape, anchors, num_classes):
    n = len(annotation_lines)
    if n==0 or batch_size<=0: return None
    return data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes)

def create_tiny_model(input_shape, anchors, num_classes, load_pretrained=True, freeze_body=2,
            weights_path='model_data/yolo-tiny.h5'):
    '''create the training model, for Tiny YOLOv3'''
    K.clear_session() # get a new session
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)

    y_true = [Input(shape=(h//{0:32, 1:16}[l], w//{0:32, 1:16}[l], \
        num_anchors//2, num_classes+5)) for l in range(2)]

    model_body = tiny_yolo_body(image_input, num_anchors//2, num_classes)
    print('Create Tiny YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))

    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
        print('Load weights {}.'.format(weights_path))
        if freeze_body in [1, 2]:
            # Freeze the darknet body or freeze all but 2 output layers.
            num = (20, len(model_body.layers)-2)[freeze_body-1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))

    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.7})(
        [*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss)

    return model

In [3]:
annotation_path = 'model_data/train_coco.txt'
log_dir = 'logs/000/'
classes_path = 'model_data/coco_classes.txt'
anchors_path = 'model_data/yolo_anchors.txt'
class_names = get_classes(classes_path)
num_classes = len(class_names)
anchors = get_anchors(anchors_path)

input_shape = (416,416) # multiple of 32, hw

is_tiny_version = len(anchors)==6 # default setting
if is_tiny_version:
    model = create_tiny_model(input_shape, anchors, num_classes,        
                              freeze_body=2, weights_path='model_data/downloaded_coco_tiny.h5')
else:
    model = create_model(input_shape, anchors, num_classes,
                         freeze_body=2, weights_path='model_data/downloaded_coco.h5') # make sure you know what you freeze



logging = TensorBoard(log_dir=log_dir)
checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
                             monitor='val_loss', save_weights_only=True, save_best_only=True, period=3)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)


with open(annotation_path) as f:
    lines = f.readlines()

#lines = lines[:10000]
print ("Training data size: ", len(lines))
print(lines[0])

np.random.seed(10101)
np.random.shuffle(lines)
np.random.seed(None)
val_split = 0.1
num_val = int(len(lines)*val_split)
num_train = len(lines) - num_val

# add gpu growth flags
#tf_config.gpu_options.allow_growth = True
#tf_config.gpu_options.per_process_gpu_memory_fraction = 0.1
#model = multi_gpu_model(model, gpus=2)

if True:
    model.compile(optimizer=Adam(lr=1e-3), loss={
            # use custom yolo_loss Lambda layer.
         'yolo_loss': lambda y_true, y_pred: y_pred})

    batch_size = 8
    print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
    model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
            steps_per_epoch=max(1, num_train//batch_size),
            validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
            validation_steps=max(1, num_val//batch_size),
            epochs=50,
            initial_epoch=0,
            callbacks=[logging, checkpoint])
    model.save_weights(log_dir + 'trained_weights_stage_1.h5')

    # Unfreeze and continue training, to fine-tune.
    # Train longer if the result is not good.
if True:
    for i in range(len(model.layers)):
        model.layers[i].trainable = True
    model.compile(optimizer=Adam(lr=1e-4), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) # recompile to apply the change
    print('Unfreeze all of the layers.')

    batch_size = 8 # note that more GPU memory is required after unfreezing the body
    print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
    model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
        steps_per_epoch=max(1, num_train//batch_size),
        validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
        validation_steps=max(1, num_val//batch_size),
        epochs=100,
        initial_epoch=50,
        callbacks=[logging, checkpoint, reduce_lr, early_stopping])
    model.save_weights(log_dir + 'trained_weights_final.h5')


Create YOLOv3 model with 9 anchors and 80 classes.
Load weights model_data/downloaded_coco.h5.
Freeze the first -3 layers of total 252 layers.
Training data size:  82080
model_data/coco/images/train2014/COCO_train2014_000000480023.jpg 116,305,401,571,52 75,134,278,349,0 239,176,349,309,0 394,222,479,419,0 0,0,78,164,0 345,217,407,284,0 4,128,124,286,0 381,241,430,298,0 225,111,292,201,0 45,402,466,635,0 0,525,68,639,0 0,236,126,518,0

Train on 73872 samples, val on 8208 samples, with batch size 8.
Epoch 1/50
9234/9234 [==============================] - 4810s 521ms/step - loss: 89.1911 - val_loss: 82.1989
Epoch 2/50
9234/9234 [==============================] - 4788s 518ms/step - loss: 79.2686 - val_loss: 77.3676
Epoch 3/50
9234/9234 [==============================] - 4807s 521ms/step - loss: 73.0290 - val_loss: 71.4181
Epoch 4/50
9234/9234 [==============================] - 4789s 519ms/step - loss: 69.9158 - val_loss: 71.8447
Epoch 5/50
9234/9234 [==============================] - 4790s

Epoch 70/100
9234/9234 [==============================] - 4825s 523ms/step - loss: 54.6709 - val_loss: 54.3391
Epoch 71/100
9234/9234 [==============================] - 4818s 522ms/step - loss: 54.5049 - val_loss: 54.3035
Epoch 72/100
9234/9234 [==============================] - 4825s 522ms/step - loss: 54.3289 - val_loss: 54.3433
Epoch 73/100
9234/9234 [==============================] - 4823s 522ms/step - loss: 54.4074 - val_loss: 54.2569
Epoch 74/100
9234/9234 [==============================] - 4823s 522ms/step - loss: 54.2760 - val_loss: 54.2430
Epoch 75/100
9234/9234 [==============================] - 4824s 522ms/step - loss: 54.1004 - val_loss: 53.9496
Epoch 76/100
9234/9234 [==============================] - 4818s 522ms/step - loss: 54.1495 - val_loss: 54.2083
Epoch 77/100
9234/9234 [==============================] - 4823s 522ms/step - loss: 54.1353 - val_loss: 53.7250
Epoch 78/100
9234/9234 [==============================] - 4823s 522ms/step - loss: 54.0204 - val_loss: 53.9343
E